In [ ]:
import numpy as np
import pandas as pd
import os
import sqlite3
import pynndescent
import umap
import gzip
from datetime import date
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
import sklearn.cluster as cluster
from math import sqrt

from bokeh.plotting import *
from bokeh.layouts import column, row
from bokeh.transform import factor_cmap
import bokeh.palettes as palettes
from bokeh.models import *
from bokeh.models.tools import *

import sys; 
sys.path.insert(0, '/covid-vis/scripts')

from utils import id_to_path, media_tpl_to_path

In [ ]:
output_notebook()

In [ ]:
#tweetIDs.txt contains all tweetIds, tweetsdb must be reconstructed.
db = sqlite3.connect('/covid-vis/tweets.db')

# Compile Data

In [ ]:
def load_npygz(media_id):
    path = os.path.join(id_to_path(media_id), str(media_id))
    try:
        file = gzip.open(path + '-fc7.npy.gz', 'rb')
    except FileNotFoundError as e:
        print(e)
        try:
            file = gzip.open(path + '.npy.gz', 'rb')
        except FileNotFoundError as e:
            print('FAILED', e)
            return None

    arr = np.load(file)
    file.close()
    return arr

In [ ]:
df_all = pd.read_sql_query("select distinct(media_id), url from resources where media_id is not null order by media_id", db)

In [ ]:
fc7_all = [load_npygz(media_id) for media_id in df_all['media_id']]
fc7_all = [x.mean(0) if x.shape == (10, 4096) else x for x in fc7_all]

In [ ]:
df_all['fc7'] = fc7_all


In [ ]:
df_nonull = df_all[~df_all.fc7.isnull()]
df_nonull['fc7_hash'] = df_nonull['fc7'].apply(lambda x: hash(tuple(x)))

In [ ]:
df_unique = df_nonull.drop_duplicates(subset=['fc7_hash'])

In [ ]:
fc7_unique = np.stack(df_unique['fc7'])

# UMAP

In [ ]:
fc7_embed_100 = umap.UMAP(metric='cosine', min_dist=0.01, n_neighbors=20, n_components=100).fit_transform(fc7_unique)

In [ ]:
fc7_proj_2 = umap.UMAP(metric='cosine', min_dist=0.01, n_neighbors=20, n_components=2).fit_transform(fc7_unique)

# K-Means

In [ ]:
for k in [5, 10, 15, 20, 30, 40]:
    df_unique['k' + k] = cluster.KMeans(n_clusters=k).fit_predict(fc7_embed_100)

# Dashboard

In [ ]:
def vis(df, title='', cluster='k40'):
    
    df = df.copy().rename(
        columns={0: 'x', 1: 'y'}
    ).astype({
        'k5': 'str', 'k10': 'str', 'k15': 'str', 
        'k20': 'str', 'k30': 'str', 'k40': 'str', 
        'media_id': 'str'
    })
    
    pipeline = Pipeline([
#         ('bins', KBinsDiscretizer(n_bins=5)),
        ('minmax', MinMaxScaler(feature_range=(3, 50)))
    ])
    engagement_sqrt = np.array([sqrt(x) for x in df['engagement_count']]).reshape(-1, 1)
    df['engagement_scaled'] = pipeline.fit_transform(engagement_sqrt)
    
    source = ColumnDataSource(df)
    filtered = ColumnDataSource(df)
    
    p = figure(
        title=title + ' ' + cluster, 
        tools='tap,pan,wheel_zoom,box_zoom,zoom_in,zoom_out,reset,save',
        x_range=(df['x'].min(), df['x'].max()),
        y_range=(df['y'].min(), df['y'].max())
    )

    cluster_color = factor_cmap(cluster, 
                                palette=palettes.Category20_20 + palettes.Category20b_20, 
                                factors=[str(x) for x in range(0, 40)]
                               )
    
    circles = p.circle(source=filtered,
                       x='x', 
                       y='y', 
                       fill_color=cluster_color,
                       size='engagement_scaled',

                       fill_alpha=0.5,
                       line_alpha=0.2,
                       line_color='white',

                       selection_fill_alpha=1,
                       selection_line_color='#666666',
                       selection_line_alpha=1,

                       nonselection_fill_alpha=0.1
    #                    legend_group='cluster',
    #                    size=2
                      )

    hover = HoverTool(
        tooltips='''
            <div style="width: 500px">
                <p>@media_id, K20: @k20, K30: @k30, K40: @k40</p>

                <div style="display: flex;">
                    <div style="flex: 1;">
                        <img src=@url style="width: 100%">
                    </div>

                    <div style="flex: 1; padding-left: 5px;">
                        <p>@full_text</p>
                        <p>&mdash; @user_name, @created_at{%F %R}</p>
                        <p style="font-size: x-small;">
                            @reply_count{0,} replies<br> 
                            @favorite_count{0,} favs<br>
                            @quote_count{0,} quotes<br>
                            @retweet_count{0,} RTs
                        </p>
                    </div>
                </div>
            </div>
        ''',
        formatters={
            '@created_at': 'datetime'
        }
    )
    p.add_tools(hover)

    div = Div(name = 'tooltip',
             text = '''<div id="tooltip" 
                         style="position: absolute; 
                                 display: none; 
                                 z-index: 100; 
                                 background: white; 
                                 border: 1px solid grey; 
                                 padding: 5px;">
                        </div>''')

    p.select(TapTool).callback = CustomJS(args = {'circles': circles, 
                                                  'plot': p, 
                                                  'tp': hover.tooltips}, 
                                          code = '''  
        if (cb_data.source.selected.indices.length > 0) {
            var selected_index = cb_data.source.selected.indices[0];
            var tooltip = document.getElementById("tooltip");

            tooltip.style.display = 'block';
            tooltip.style.left = Number(cb_data.geometries.sx) + 'px';
            tooltip.style.top = Number(cb_data.geometries.sy) + 'px';

            tp = tp.replace('@media_id', cb_data.source.data.media_id[selected_index]);
            tp = tp.replace('@k20', cb_data.source.data.k20[selected_index]);
            tp = tp.replace('@k30', cb_data.source.data.k30[selected_index]);
            tp = tp.replace('@k40', cb_data.source.data.k40[selected_index]);
            tp = tp.replace('@url', cb_data.source.data.url[selected_index]);
            tp = tp.replace('@full_text', cb_data.source.data.full_text[selected_index]);
            tp = tp.replace('@user_name', cb_data.source.data.user_name[selected_index]);
            tp = tp.replace('@created_at{%F %R}', new Date(cb_data.source.data.created_at[selected_index]));
            tp = tp.replace('@reply_count{0,}', cb_data.source.data.reply_count[selected_index]);
            tp = tp.replace('@favorite_count{0,}', cb_data.source.data.favorite_count[selected_index]);
            tp = tp.replace('@quote_count{0,}', cb_data.source.data.quote_count[selected_index]);
            tp = tp.replace('@retweet_count{0,}', cb_data.source.data.retweet_count[selected_index]);

            tooltip.innerHTML = tp;
        } else {
            document.getElementById("tooltip").style.display = \"none\"
        }''')

    filtered.selected.js_on_change('indices', 
                                 CustomJS(code = 'if (cb_obj.indices.length == 0) document.getElementById("tooltip").style.display = \"none\"'))

    start_date = date(2020, 1, 1)
    end_date = df['created_at'].max()
    daterange = DateRangeSlider(start=start_date, end=end_date, 
                                value=(start_date, end_date),
                                step=1
                               )
    daterange.js_on_change('value', CustomJS(args=dict(source=source, filtered=filtered), code='''
        const start_date = cb_obj.value[0];
        const end_date   = cb_obj.value[1];
        const cols = Object.keys(source.data);

        // Empty source dataset
        for (let c of cols) filtered.data[c] = [];

        // Repopulate based on ref
        source.data.created_at.forEach(function(d, i) {
            if (d >= start_date && d <= end_date)
                for (let c of cols) filtered.data[c].push(source.data[c][i])
        });

        filtered.change.emit()
    '''))

    return Column(Row(div, p), daterange)

In [ ]:
dfc7 = pd.concat([df_unique, pd.DataFrame(fc7_proj_2)], axis=1)

In [ ]:
df_tweets = pd.read_sql_query('''select resources.media_id, resources.tweet_id, tweets.*
                                from resources, tweets
                                where resources.tweet_id = tweets.id
                                and resources.mark is not null''', db, parse_dates=['created_at'])

In [ ]:
df_joined = pd.merge(dfc7, df_tweets, on=['media_id'])

In [ ]:
df_joined = df_joined.fillna({'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0})

In [ ]:
df_joined['engagement_count'] = 1 + df_joined['quote_count'] + df_joined['reply_count'] + df_joined['retweet_count'] + df_joined['favorite_count']

In [ ]:
df_joined_head = df_joined.sort_values(by=['engagement_count'], ascending=False).groupby('media_id').head(1)

In [ ]:
show(vis(df_joined_head))